In [1]:
%pip install wandb datasets transformers trl torch peft huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [2]:
import wandb
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling
from trl import SFTTrainer, SFTConfig
import torch
from peft import LoraConfig, get_peft_model

In [3]:
# !huggingface-cli login

In [4]:
# 모델과 토크나이저 로드
model_name = "gpt2-large"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

# corpus.json 파일 읽기
with open('./data/corpus.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# train/test 분리 (80:20 비율)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

print(f"전체 데이터 크기: {len(data)}")
print(f"학습 데이터 크기: {len(train_data)}")
print(f"테스트 데이터 크기: {len(test_data)}")

# 데이터 예시 출력
print("\n=== 학습 데이터 예시 ===")
print(train_data[0])
print("\n=== 테스트 데이터 예시 ===")
print(test_data[0])

# 리스트를 Dataset 형식으로 변환
train_dataset = Dataset.from_list(train_data)
test_dataset = Dataset.from_list(test_data)

전체 데이터 크기: 125
학습 데이터 크기: 100
테스트 데이터 크기: 25

=== 학습 데이터 예시 ===
{'input': '감정: 혼란스러움 / 원하는 효과: 위로받고 싶다 / 직업: 구직자 / 현재 상황: 스트레스 해소 / 집중도: 빠르게 몰입할 수 있는 책', 'instruction': '혼란스러운 심리 상태에서 위안을 얻고 싶을 때 적절한 3권을 골라, 추천 이유와 효과를 적으세요.', 'output': '1. 마음은 괜찮냐고 시가 물었다 - 황인환\n\n* 추천 이유: 시를 통해 감정의 혼란을 천천히 정돈할 수 있습니다.\n* 이 책이 도움이 될 수 있는 이유: 짧은 글이 매일의 위로가 되어줄 수 있습니다.\n\n2. 오렌지와 빵칼 - 청예\n\n* 추천 이유: 다양한 인물의 시선을 따라가며 자신과 상황을 분리해 생각할 수 있습니다.\n* 이 책이 도움이 될 수 있는 이유: 쉽게 몰입할 수 있는 이야기로 감정적 환기를 제공합니다.\n\n3. 이렇게 누워만 있어도 괜찮을까 - 안예슬\n\n* 추천 이유: 혼란스러운 변화 속에서 자신에게 더 관대해지는 법을 일깨워 줍니다.\n* 이 책이 도움이 될 수 있는 이유: 편안한 휴식의 순간을 통해 스스로 회복력을 느낄 수 있습니다.'}

=== 테스트 데이터 예시 ===
{'instruction': '슬럼프에 빠진 운동선수나 직장인에게 자기 극복과 성장 동기를 줄 책을 추천해줘.', 'input': '감정: 슬럼프, 무기력\n직업: 운동선수, 직장인\n효과: 자기 극복, 동기부여\n상황: 경기 또는 업무 후\n집중도: 몰입감 높은 책', 'output': '1. 운동의 진실과 기쁨 - 강윤규 지음\n   - 추천 이유: 운동을 통한 자기성찰과 회복의 과정이 담긴 책입니다.\n   - 이 책이 도움이 될 수 있는 이유: 운동의 본질적 기쁨과 극복 경험을 통해 재충전할 수 있습니다.\n2. 해나가는 힘 : 늘 사랑받고 싶은 사람을 위한 심리학 - 이승현\n   - 추천 이유: 자신을 사랑하고 자신감과 추진력

In [6]:
# 프롬프트 포매팅 함수 정의
def formatting_func(example):
    # instruction, input, output이 단일 문자열인 경우를 처리
    instruction = example['instruction']
    input_text = example['input'].strip() if example['input'] else ""
    output = example['output']

    # 형식화된 프롬프트 생성
    text = f"[Instruction]\n{instruction.strip()}\n\n"
    if input_text:
        text += f"[Input]\n{input_text}\n\n"
    text += f"[Output]\n{output.strip()}"

    return text

# 데이터 콜레이터 설정
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Causal Language Modeling을 위해 False로 설정
)

In [7]:
wandb.init(
    project='Hanghae99-book-recommandation',
    name=f'gpt-finetuning',
    reinit=True
)

model = AutoModelForCausalLM.from_pretrained(model_name)

trainer = SFTTrainer(
    model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    args=SFTConfig(
        output_dir=f"/tmp/clm-instruction-tuning",
        max_seq_length=128,
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=3,
        learning_rate=2e-5,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss"
    ),
    formatting_func=formatting_func,
    data_collator=data_collator,
)

# 학습 시작
train_result = trainer.train()
metrics = train_result.metrics
trainer.log_metrics("train", metrics)

trainer.save_model()

wandb.finish()

wandb: Currently logged in as: knospe1 (knospe1-gaeun) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Applying formatting function to train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/100 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

...

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,1.525300,1.117897
2,1.082500,0.994548
3,0.927600,0.973069


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


***** train metrics *****
  total_flos               =   152004GF
  train_loss               =     1.1119
  train_runtime            = 5:04:07.42
  train_samples_per_second =      0.016
  train_steps_per_second   =      0.002


eval/loss,█▂▁
eval/mean_token_accuracy,▁██
eval/num_tokens,▁▅█
eval/runtime,▄█▁
eval/samples_per_second,▃▁█
eval/steps_per_second,▃▁█
train/epoch,▁▂▃▅▆██
train/global_step,▁▂▃▅▆██
train/grad_norm,▆█▁
train/learning_rate,█▄▁
train/loss,█▃▁


### LoRA 적용

In [10]:
def train_with_lora(lora_r, model_name, train_dataset, test_dataset, formatting_func, data_collator):
    # 디바이스 자동 선택: CUDA > MPS > CPU
    if torch.cuda.is_available():
        device = torch.device("cuda")
        torch_dtype = torch.float16  # CUDA에서는 float16이 빠르고 효율적
        use_fp16 = True
        device_map = 'auto'
        print("CUDA를 사용합니다.")
    elif getattr(torch.backends, 'mps', None) and torch.backends.mps.is_available():
        device = torch.device("mps")
        torch_dtype = torch.float32  # MPS에서는 float32가 안전함
        use_fp16 = False
        device_map = None
        print("MPS를 사용합니다.")
    else:
        device = torch.device("cpu")
        torch_dtype = torch.float32
        use_fp16 = False
        device_map = None
        print("CPU를 사용합니다.")

    wandb.init(
        project='Hanghae99-book-recommandation',
        name=f'gpt-finetuning-with-lora-r{lora_r}',
        reinit=True
    )

    print(f"\n=== Training with LoRA rank {lora_r} ===")

    # 모델 로드 (device_map이 있을 때만 넣음)
    model_kwargs = {
        "torch_dtype": torch_dtype
    }
    if device_map is not None:
        model_kwargs["device_map"] = device_map
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        **model_kwargs
    )

    # device_map이 없으면 명시적으로 모델 이동
    if device_map is None:
        model = model.to(device)

    # LoRA 설정
    lora_config = LoraConfig(
        r=lora_r,
        lora_alpha=32,
        target_modules=["c_attn", "c_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()

    # SFTTrainer 정의
    trainer = SFTTrainer(
        model,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        args=SFTConfig(
            output_dir=f"/tmp/clm-instruction-tuning-lora-{lora_r}",
            max_seq_length=128,
            eval_strategy="epoch",
            save_strategy="epoch",
            logging_steps=10,
            num_train_epochs=3,
            learning_rate=2e-5,
            load_best_model_at_end=True,
            metric_for_best_model="eval_loss",
            per_device_train_batch_size=1,  # MPS/CPU 환경은 1, CUDA는 4까지도 가능(필요시 조건 분기)
            gradient_accumulation_steps=4,
            fp16=use_fp16
        ),
        formatting_func=formatting_func,
        data_collator=data_collator,
    )

    # 모델이 올바른 device로 있는지 추가 확인 필요 시 trainer.model = trainer.model.to(device)
    if device_map is None:
        trainer.model = trainer.model.to(device)

    # 학습 시작
    train_result = trainer.train()
    metrics = train_result.metrics
    trainer.log_metrics("train", metrics)

    # 모델 저장
    trainer.save_model()

    wandb.finish()

In [ ]:
train_with_lora(
    lora_r=8,
    model_name=model_name,
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    formatting_func=formatting_func,
    data_collator=data_collator
)

MPS를 사용합니다.


python(80707) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



=== Training with LoRA rank 8 ===


/opt/anaconda3/envs/llm/lib/python3.12/site-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 4,055,040 || all params: 778,085,120 || trainable%: 0.5212


...

Adding EOS to eval dataset:   0%|          | 0/25 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/25 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/25 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
